In [1]:
# I created this to look at available stocks vs short interest stocks for certain companies

In [2]:
import pandas as pd
from IPython.core.display import display, HTML
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import scipy as sp
from scipy import stats

In [3]:
shorts_1 = "https://www.highshortinterest.com/"
##shorts_2 = "https://www.highshortinterest.com/all/2" ## make sure to unhastag this if 2 pages

In [4]:
r_1 = requests.get(shorts_1)

shorts_tables_1 = pd.read_html(r_1.text, header=0) # to put the tables into pandas form

##r_2 = requests.get(shorts_2)  ## make sure to unhastag this if 2 pages
##shorts_tables_2 = pd.read_html(r_2.text, header=0)  ## make sure to unhastag this if 2 pages

In [5]:
shorts1 = shorts_tables_1[2]
##shorts2 = shorts_tables_2[2]  ## make sure to unhastag this if 2 pages

In [39]:
for_str = shorts_tables_1[0].iloc[2].to_string() # to extract the other info besides dataframe and change it to strings
#iloc change change for ranging from 0 to 2. Make sure check

In [40]:
type(for_str)

str

In [41]:
for_str

'High Short Interest Stocks HighShortInterest.com provides a convenient sorted database of stocks which have a short interest of over 20 percent. Additional key data such as the float, number of outstanding shares, and company industry is displayed. Data is presented for the Nasdaq Stock Market, the New York Stock Exchange, and the American Stock Exchange. You can view the data for all exchanges together or only view exchanges of interest by clicking on the appropriate tab. Stocks with high short interest are often very volatile and are well known for making explosive upside moves (known as a short squeeze). Stock traders will often flock to such stocks for no reason other than the fact that they have a high short interest and the price can potentially move up very quickly as traders with open short positions move to cover. Disclaimer: We have made our best effort to provide accurate data, but always do your own research and double check the information presented here before making an 

In [42]:
#Using regular expressions, you can use re.finditer to find all (non-overlapping) occurences:

import re
for m in re.finditer('on', for_str):
        print('on found', m.start(), m.end())
a = m.end() + 1 # 1 is to add after the on

on found 61 63
on found 151 153
on found 410 412
on found 454 456
on found 672 674
on found 816 818
on found 964 966
on found 1036 1038


In [43]:
date = for_str[a:] # to get the date in the string, last few letters
date_obj = datetime.strptime(date, '%B %d, %Y' )

In [45]:
type(date_obj)

datetime.datetime

# shorts = pd.concat([shorts1, shorts2], ignore_index=True) #to combine both dataframes ## only 1 page for february 25

In [46]:
shorts = shorts1 # if one page use this

In [47]:
null_data = shorts[shorts.isnull().any(axis=1)]

In [48]:
shorts.drop(null_data.index,inplace=True) # drop rows containing NaN

In [49]:
shorts.reset_index(inplace=True, drop=True) # reset index

In [50]:
tic = shorts['Ticker']
com = shorts['Company']
new = []

In [51]:
for tick in tic:
    if len(tick) > 4:
        new.append(tick)
#here, if the length of the ticker is more than 4, it means it is not a stock, see below for details

In [52]:
new

['<!-- google_ad_client = "pub-1641527371507802"; /* Financial */ google_ad_slot = "4718314016"; google_ad_width = 728; google_ad_height = 90; //-->']

In [53]:
drop = shorts.loc[shorts['Ticker'] == new[0]] # to find rows with google information which we do not need
# loc() is label based data selecting method

In [54]:
drop # shows which rows is not needed

,Ticker,Company,Exchange,ShortInt,Float,Outstd,Industry
25,"<!-- google_ad_client = ""pub-1641527371507802""...","<!-- google_ad_client = ""pub-1641527371507802""...","<!-- google_ad_client = ""pub-1641527371507802""...","<!-- google_ad_client = ""pub-1641527371507802""...","<!-- google_ad_client = ""pub-1641527371507802""...","<!-- google_ad_client = ""pub-1641527371507802""...","<!-- google_ad_client = ""pub-1641527371507802""..."


In [55]:
shorts.drop(drop.index,inplace=True) # drop rows containing google information which we do not need

In [56]:
shorts.reset_index(inplace=True, drop=True) 

In [57]:
shorts.drop(columns=['Ticker','Industry','Exchange'], inplace=True)

In [58]:
shorts.rename(columns={'ShortInt':'Short Int (%)','Float':'Float (M)', 'Outstd':'Outstd (M)'},inplace=True)

In [59]:
shorts['Short Int (%)'] = shorts['Short Int (%)'].str.rstrip('%')
shorts['Float (M)'] = shorts['Float (M)'].str.rstrip('M')
shorts['Outstd (M)'] = shorts['Outstd (M)'].str.rstrip('M')

# Series.str.strip - Remove leading and trailing characters in Series/Index.
# Series.str.lstrip - Remove leading characters in Series/Index.
# Series.str.rstrip - Remove trailing characters in Series/Index.

In [60]:
shorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Company        41 non-null     object
 1   Short Int (%)  41 non-null     object
 2   Float (M)      41 non-null     object
 3   Outstd (M)     41 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [61]:
shorts.iloc[[31,32]] # iloc takes index as argument / indexed based selecting method

,Company,Short Int (%),Float (M),Outstd (M)
31,GTT Communications Inc,21.74,45.75,58.82
32,Senseonics Holdings Inc,21.73,333.73,427.91


In [62]:
shorts[["Short Int (%)", "Float (M)", "Outstd (M)"]] = shorts[["Short Int (%)", "Float (M)", "Outstd (M)"]].apply(pd.to_numeric)
# this will change the columns from strings to float

In [63]:
shorts.info() # confirmed that columns change to float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        41 non-null     object 
 1   Short Int (%)  41 non-null     float64
 2   Float (M)      41 non-null     float64
 3   Outstd (M)     41 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.4+ KB


In [64]:
shorts['Shr avbl (%)'] = round(shorts['Float (M)']/shorts['Outstd (M)'] * 100,2) # new column for available shares % 

In [65]:
shorts['Diff'] = round(shorts['Shr avbl (%)'] - shorts['Short Int (%)'],2) # new column for diff between shares

In [66]:
buy = []
for value in shorts['Diff']:
    if value <= 0:
        buy.append('Yes')
    else:
        buy.append('No')
shorts['Buy?'] = buy
# to evaluate whether the Diff column is +ve or -ve. If -ve, will consider buy and vice versa

In [67]:
yes_buy = shorts[shorts.eq("Yes").any(1)] # to find yes located in anywhere in dataframe

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [68]:
yes_buy.sort_values(by=['Diff', 'Shr avbl (%)'])

,Company,Short Int (%),Float (M),Outstd (M),Shr avbl (%),Diff,Buy?
14,Revlon Inc,27.13,6.84,53.49,12.79,-14.34,Yes
0,Root Inc,41.28,35.02,86.89,40.30,-0.98,Yes


In [69]:
with pd.ExcelWriter('Shorted Stock Interest 2021.xlsx', mode='a',engine='openpyxl') as writer:  
    shorts.to_excel(writer, sheet_name = f'Shorts {date_obj.strftime("%B %d, %Y")}')
    shorts.describe().to_excel(writer, sheet_name = f'Shorts Stats {date_obj.strftime("%B %d, %Y")}')
    yes_buy.to_excel(writer, sheet_name=f'Buy(Yes) {date_obj.strftime("%B %d, %Y")}')
    yes_buy.describe().to_excel(writer, sheet_name = f'Buy(Yes) Stats {date_obj.strftime("%B %d, %Y")}')
# able to append to existing excel document with new sheet

In [ ]:
# THINK ABOUT DOING A BELL CURVE FOR BUY (YES) & ALL DATA

In [ ]:
graph_yes = yes_buy['Short Int (%)'].describe()[3:8]

In [ ]:
## plotting the data
plt.hist(graph_yes, bins=100)
plt.show()